In [1]:
# Optimizacion Bayesiana de hiperparametros de  rpart
# que va directamente contra el Public Leaderboard
# este script AUN no entrena en un dataset con oversampling de los BAJA+2

# dedicado a Federico Idoeta, Impossible is Nothing,  02-sep-2022


# limpio la memoria
rm(list = ls()) # remove all objects
gc() # garbage collection

require("data.table")
require("rlist")

require("rpart")
require("parallel")

# paquetes necesarios para la Bayesian Optimization
require("DiceKriging")
require("mlrMBO")


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,627750,33.6,1404611,75.1,1024906,54.8
Vcells,1150018,8.8,8388608,64.0,1769151,13.5


Loading required package: data.table

Loading required package: rlist

Loading required package: rpart

Loading required package: parallel

Loading required package: DiceKriging

Loading required package: mlrMBO

Loading required package: mlr

Loading required package: ParamHelpers

Warning message: 'mlr' is in 'maintenance-only' mode since July 2019.
Future development will only happen in 'mlr3'
(<https://mlr3.mlr-org.com>). Due to the focus on 'mlr3' there might be
uncaught bugs meanwhile in {mlr} - please consider switching.

Loading required package: smoof

Loading required package: checkmate


Attaching package: 'checkmate'


The following object is masked from 'package:DiceKriging':

    checkNames




In [3]:
# Defino la  Optimizacion Bayesiana
PARAM <- list()
PARAM$experimento <- "HT3990_correcto"

# cantidad de iteraciones de la Optimizacion Bayesiana
PARAM$BO_iter <- 24  # iteraciones inteligentes   24= 40 - 4*4

#  de los hiperparametros
PARAM$hs <- makeParamSet(
  makeIntegerParam("minsplit", lower = 500L, upper = 1500L),
  makeIntegerParam("minbucket", lower = 200L, upper = 800L),
  makeIntegerParam("maxdepth", lower = 6L, upper = 12L),
  makeIntegerParam("corte", lower = 8000L, upper = 10000L),
  forbidden = quote(minbucket > 0.5 * minsplit)
)
# minbuket NO PUEDE ser mayor que la mitad de minsplit

PARAM$semilla_azar <- 364747 # primer semilla de Federico

In [4]:
#------------------------------------------------------------------------------

leer_numero <- function( mensaje ) {
  res <- readline( mensaje )
  while( is.na( as.numeric( res ))) {
    cat( "Debe introducit un numero, el separador decimal es la coma\n" )
    res <- readline( mensaje )
  }

  return( as.numeric(res) )
}
#------------------------------------------------------------------------------


In [5]:
leer_verificado <- function( mensaje ) {
  repeat {
  
    n1 <- leer_numero( mensaje )
    cat( "Por favor, vuelva a cargar el mismo numero\n" )
    n2 <- leer_numero( mensaje )

   if( n1 != n2 )  cat( "Los numeros no coinciden, debe volver a cargar\n\n" )
   if( n1== n2 ) break
  }

  return( n1 )
}

In [8]:
#------------------------------------------------------------------------------
# graba a un archivo los componentes de lista
# para el primer registro, escribe antes los titulos

loguear <- function(reg, arch = NA, folder = "./work/", ext = ".txt",
                    verbose = TRUE) {
  archivo <- arch
  if (is.na(arch)) archivo <- paste0(folder, substitute(reg), ext)

  # Escribo los titulos
  if (!file.exists(archivo)) {
    linea <- paste0(
      "fecha\t",
      paste(list.names(reg), collapse = "\t"), "\n"
    )

    cat(linea, file = archivo)
  }

  # la fecha y hora
  linea <- paste0(
    format(Sys.time(), "%Y%m%d %H%M%S"), "\t",
    gsub(", ", "\t", toString(reg)), "\n"
  )

  # grabo al archivo
  cat(linea, file = archivo, append = TRUE)

  # imprimo por pantalla
  if (verbose) cat(linea)
}

In [9]:
#----------------------------------------------------------------------------
# param tiene los hiperparametros del arbol

ArbolSimple <- function( data, param, iteracion) {

  param2 <- copy( param )
  param2$cp <- -1
  param2$minsplit <- param$minsplit 
  param2$minbucket <- param$minbucket
  param2$corte <- param$corte

  modelo <- rpart("clase_binaria ~ . - clase_ternaria",
    data = dtrain,
    xval = 0,
    control = param2,
    weights = pesos
  )

  # aplico el modelo a los datos de testing
  # aplico el modelo sobre los datos de testing
  # quiero que me devuelva probabilidades
  prediccion <- predict(modelo,
    dapply,
    type = "prob"
  )

  # esta es la probabilidad de baja
  prob_baja <- prediccion[, "POS"]

  tablita <- copy( dapply[, list(numero_de_cliente) ] )
  tablita[ , prob := prob_baja ]
  setorder( tablita, -prob )

  # grabo el submit a Kaggle
  tablita[ , Predicted := 0L ]
  tablita[ 1:param2$corte, Predicted := 1L ]

  nom_submit <- paste0("z399_", sprintf( "%03d", iteracion ), ".csv" )
  fwrite( tablita[ , list(numero_de_cliente, Predicted)],
          file= nom_submit,
          sep= "," )

  # solicito que el humano a cargo ingrese la ganancia publica
  mensaje <- paste0( "haga el submit a Kaggle de ", nom_submit,
                     " y cargue la ganancia publica : " )

  ganancia_public <- leer_verificado( mensaje )

  return(ganancia_public)
}
#------------------------------------------------------------------------------

In [10]:
# esta funcion solo puede recibir los parametros que se estan optimizando
# el resto de los parametros, lamentablemente se pasan como variables globales

EstimarGanancia <- function(x) {
  GLOBAL_iteracion <<- GLOBAL_iteracion + 1

  # x los hiperparametros del arbol
  ganancia_public <- ArbolSimple( dtrain, x, GLOBAL_iteracion )

  # logueo
  xx <- x
  xx$cp <- -1
  xx$ganancia <- ganancia_public
  xx$iteracion <- GLOBAL_iteracion
  loguear(xx, arch = archivo_log)

  # para que mlrMBO tenga todo reseteado
  set.seed( PARAM$semilla_azar )

  return(ganancia_public)
}

In [11]:
# Aqui empieza el programa

# Establezco el Working Directory
setwd("C:\\Users\\flore\\Desktop\\dmef\\datasets\\")


In [12]:
# cargo los datos
dataset <- fread("./competencia_01_clase_ternaria.csv")
dataset <- subset(dataset, select = -count)

In [13]:
dim(dataset)

[1] 491063    155

In [14]:
# defino la clase_binaria2
dataset[ , clase_binaria := ifelse( clase_ternaria=="CONTINUA", "NEG", "POS" ) ]

dtrain <- dataset[foto_mes==202103]
dapply <- dataset[foto_mes==202105]

# definicion vector de pesos para oversampling
pesos <- copy( dtrain[, ifelse( clase_ternaria=="CONTINUA",   1.0, 100.0  ) ] )

In [15]:
# creo la carpeta donde va el experimento
#  HT  representa  Hiperparameter Tuning
dir.create("./exp/", showWarnings = FALSE)
dir.create( paste0("./exp/", PARAM$experimento, "/"), 
           showWarnings = FALSE)

# Establezco el Working Directory DEL EXPERIMENTO
setwd( paste0("./exp/", PARAM$experimento, "/") )


# en estos archivos quedan los resultados
archivo_log <- "BO_log_OS.txt"
archivo_BO <- "bayesian_OS.RDATA"


In [16]:
# leo si ya existe el log
#  para retomar en caso que se se corte el programa
GLOBAL_iteracion <- 0

if (file.exists(archivo_log)) {
  tabla_log <- fread(archivo_log)
  GLOBAL_iteracion <- nrow(tabla_log)
}




In [17]:
# Aqui comienza la configuracion de la Bayesian Optimization

funcion_optimizar <- EstimarGanancia

configureMlr(show.learner.output = FALSE)

# configuro la busqueda bayesiana,
#  los hiperparametros que se van a optimizar
# por favor, no desesperarse por lo complejo
# minimize= FALSE estoy Maximizando la ganancia
obj.fun <- makeSingleObjectiveFunction(
  fn = funcion_optimizar,
  minimize = FALSE,
  noisy = TRUE,
  par.set = PARAM$hs,
  has.simple.signature = FALSE
)

ctrl <- makeMBOControl(
  save.on.disk.at.time = 600,
  save.file.path = archivo_BO
)

ctrl <- setMBOControlTermination(ctrl, iters = PARAM$BO_iter)
ctrl <- setMBOControlInfill(ctrl,  crit = makeMBOInfillCritEI())

surr.km <- makeLearner("regr.km",
  predict.type = "se",
  covtype = "matern3_2", control = list(trace = TRUE)
)


# para que mlrMBO tenga todo reseteado
set.seed( PARAM$semilla_azar )

# inicio la optimizacion bayesiana
if (!file.exists(archivo_BO)) {
  run <- mbo(
    fun = obj.fun,
    learner = surr.km,
    control = ctrl
  )
} else {
  run <- mboContinue(archivo_BO)
}
# retomo en caso que ya exista

Computing y column(s) for design. Not provided.



Por favor, vuelva a cargar el mismo numero
20230910 212006	911	396	9	9225	-1	64.98281	1
Por favor, vuelva a cargar el mismo numero
20230910 212041	1405	610	8	9860	-1	62.3695	2
Por favor, vuelva a cargar el mismo numero
20230910 212242	1307	650	11	8932	-1	65.47281	3
Por favor, vuelva a cargar el mismo numero
20230910 212316	773	347	11	9283	-1	65.16948	4
Por favor, vuelva a cargar el mismo numero
20230910 212353	1038	454	8	8716	-1	63.53616	5
Por favor, vuelva a cargar el mismo numero
20230910 212424	652	201	6	8571	-1	54.1329	6
Por favor, vuelva a cargar el mismo numero
20230910 212500	1459	362	7	9001	-1	60.45618	7
Por favor, vuelva a cargar el mismo numero
20230910 212537	1198	287	11	8202	-1	59.24286	8
Por favor, vuelva a cargar el mismo numero
20230910 212617	1096	467	10	9602	-1	65.37948	9
Por favor, vuelva a cargar el mismo numero
20230910 212652	1132	483	8	8019	-1	60.57285	10
Por favor, vuelva a cargar el mismo numero
20230910 212724	1232	436	6	8640	-1	56.39621	11
Por favor, vuelva a 

[mbo] 0: minsplit=911; minbucket=396; maxdepth=9; corte=9225 : y = 65 : 66.5 secs : initdesign

[mbo] 0: minsplit=1405; minbucket=610; maxdepth=8; corte=9860 : y = 62.4 : 35.2 secs : initdesign

[mbo] 0: minsplit=1307; minbucket=650; maxdepth=11; corte=8932 : y = 65.5 : 121.2 secs : initdesign

[mbo] 0: minsplit=773; minbucket=347; maxdepth=11; corte=9283 : y = 65.2 : 34.3 secs : initdesign

[mbo] 0: minsplit=1038; minbucket=454; maxdepth=8; corte=8716 : y = 63.5 : 37.0 secs : initdesign

[mbo] 0: minsplit=652; minbucket=201; maxdepth=6; corte=8571 : y = 54.1 : 31.1 secs : initdesign

[mbo] 0: minsplit=1459; minbucket=362; maxdepth=7; corte=9001 : y = 60.5 : 35.5 secs : initdesign

[mbo] 0: minsplit=1198; minbucket=287; maxdepth=11; corte=8202 : y = 59.2 : 37.4 secs : initdesign

[mbo] 0: minsplit=1096; minbucket=467; maxdepth=10; corte=9602 : y = 65.4 : 39.8 secs : initdesign

[mbo] 0: minsplit=1132; minbucket=483; maxdepth=8; corte=8019 : y = 60.6 : 35.2 secs : initdesign

[mbo] 0: m

Por favor, vuelva a cargar el mismo numero
20230910 213101	984	492	10	9696	-1	62.62616	17


[mbo] 1: minsplit=984; minbucket=492; maxdepth=10; corte=9696 : y = 62.6 : 36.1 secs : infill_ei



...

[mbo] 10: minsplit=824; minbucket=392; maxdepth=11; corte=9833 : y = 69.4 : 132.0 secs : infill_ei



Por favor, vuelva a cargar el mismo numero
20230910 214509	852	395	11	9845	-1	69.39278	27


[mbo] 11: minsplit=852; minbucket=395; maxdepth=11; corte=9845 : y = 69.4 : 79.6 secs : infill_ei



Por favor, vuelva a cargar el mismo numero
20230910 214640	842	385	10	9844	-1	69.48611	28


[mbo] 12: minsplit=842; minbucket=385; maxdepth=10; corte=9844 : y = 69.5 : 89.5 secs : infill_ei

